In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import seaborn as sn
import pandas as pd

#Files = ["nicla_N128_wing_new_format_newer_model2.txt", "nicla_N128_ring_new_format_newer_model2.txt","nicla_N128_slope_new_format_newer_model2.txt", "nicla_N128_negative_new_format_newer_model2.txt"]
Files = ["nicla_Q128_wing_new_format_newer_model2.txt", "nicla_Q128_ring_new_format_newer_model2.txt","nicla_Q128_slope_new_format_newer_model2.txt", "nicla_Q128_negative_new_format_newer_model2.txt"]
File = "nicla_N128_wing_new_format_newer_model.txt"
File = "Person detection-50-Target=0-9999-ver1.txt"
#Files = ["Person detection-50-Target=0-9999-ver1.txt", "Person detection-50-Target=1-9999-ver1.txt"]
#Files = ["Person detection-50-Target=0-9999-Quantized-ver1.txt", "Person detection-50-Target=1-9999-Quantized-ver1.txt"]
#Files = ["Test for full testing data target 0.txt", "Test for full testing data target 1.txt","Test for full testing data target 2.txt", "Test for full testing data target 3.txt"]
#File = "nicla_N128_All_Labels_new_format_newer_model.txt"
#index = 1   #the prediction that should be predicted based on the data used, 0:All_Labels, 1:ring, 2:slope
sent_variables = []
recieved_variables = []
predictions = []
predictionsPerson = []
predictionsLabelsPerson = ["Other persons", "Person 16"]
PredictionsLabels = ["Wing", "Ring", "Slope", "Nothing"]

threshold = 0.5
threshold = 0

In [ ]:
def check_serial(File): 
    counter = 1
    serialError = False
    resultsFile = open(File, "r")
    lines = resultsFile.readlines()
    for line in lines: 
        if line.startswith("Number"): 
            number = ""
            for i in line: 
                if i.isnumeric(): 
                    number += i
                elif i==",": 
                    
                    if not(counter == int(number)): 
                        serialError = True
                        print("the error is in the number at number " + str(number))
                        print("counter is " + str(counter) + "and number is " + str(number))
                        
                    counter+=1
                    break
       
        if line.startswith("Number"): 
            StartX = line.index("x=")
            StartY = line.index("y=")
            StartZ = line.index("z=")
            SplitX = line[StartX:]
            SplitY = line[StartY:]
            SplitZ = line[StartZ:]
            x,y,z = '', '', ''
            for i in SplitX: 
                if i.isnumeric(): 
                    x+=i
                if i == "," or i==".": 
                    break
            for i in SplitY: 
                if i.isnumeric(): 
                    y+=i
                if i == "," or i==".": 
                    break
            for i in SplitZ: 
                if i.isnumeric(): 
                    z+=i
                if i == "," or i==".": 
                    break
            recieved_variables.append([float(x),float(y),float(z)])
        
        if line.startswith("x="): 
            StartX = line.index("x=")
            StartY = line.index("y=")
            StartZ = line.index("z=")
            SplitX = line[StartX:]
            SplitY = line[StartY:]
            SplitZ = line[StartZ:]
            x,y,z = '', '', ''
            for i in SplitX: 
                if i.isnumeric(): 
                    x+=i
                if i == "," or i==".": 
                    break
            for i in SplitY: 
                if i.isnumeric(): 
                    y+=i
                if i == "," or i==".": 
                    break
            for i in SplitZ: 
                if i.isnumeric(): 
                    z+=i
                if i == "," or i==".": 
                    break
            sent_variables.append([round(float(x),6),round(float(y),6),round(float(z),6)])
    for idx, i in enumerate(sent_variables): 
        if (sent_variables[idx] != recieved_variables[idx]): 
            print("there was an error at " + str(idx) + " number, with variables " + str(sent_variables[idx] ) + "   " + str(recieved_variables[idx]))

    if sent_variables != recieved_variables:
        print("Sent and recieved variables have an issue")
        serialError = True
        print(len(sent_variables))
        print(len(recieved_variables))
    

        
    return serialError


In [ ]:
print(File)
if(check_serial(File)): 
    print("A datapoint has been skipped or wrongly transfered")  
else: 
    print("Serial transmission had no issues")

In [ ]:
def check_performance(File): 

    resultsFile = open(File, "r")
    lines = resultsFile.readlines()
    average_inference = 0       #average time of an inference
    max_inference = 0           #maximum inference time 
    min_inference = 99999         #minimum inference time 
    number_of_inferences = 0    #the number of times the arduino did an inference, this warries depending on the averaging of the algorithm
    for line in lines: 
        if line.startswith(" Inference time took: ") or line.startswith("Inference time took: "): 
                    number = ""
                    for i in line: 
                        if i.isnumeric(): 
                            number += i 
                   
                    average_inference +=  int(number)
                    if int(number) > max_inference: 
                        max_inference = int(number)
                    if int(number) < min_inference: 
                        min_inference = int(number)
                    
                    number_of_inferences += 1 

    average_inference = average_inference/number_of_inferences
    print("Average inference time was: " + str(average_inference) + " miliseconds")
    print("Max inference time was: " + str(max_inference) + " millsieconds, and the minimum inference time was: " + str(min_inference) + " milliseconds")

In [ ]:
def check_predictions(File, index): 

    
    resultsFile = open(File, "r")
    lines = resultsFile.readlines()

    for line in lines: 
        if line.startswith("I:"): 
            splitted = line.split()
            predictions.append([float(splitted[3]), float(splitted[7]), float(splitted[11]), float(splitted[15]), index])




In [ ]:
def check_predictionsPerson(File, index): 

    
    resultsFile = open(File, "r")
    lines = resultsFile.readlines()

    for line in lines: 
        if line.startswith("I:"): 
            splitted = line.split()
            predictionsPerson.append([float(splitted[3]), index])

In [ ]:
def get_f1_score(): 

    WW, WR, WS,WN = 0,0,0,0
    RW, RR, RS, RN = 0,0,0,0
    SW, SR, SS, SN = 0,0,0,0
    NW, NR, NS, NN = 0,0,0,0

    for i in predictions:
        if i[4]==0: 
            if i[0] > threshold: 
                WW+=1
            if i[1] > threshold: 
                WR+=1
            if i[2] > threshold: 
                WS+=1
            if i[3] > threshold: 
                WN+=1
        if i[4]==1: 
            if i[0] > threshold: 
                RW+=1
            if i[1] > threshold: 
                RR+=1
            if i[2] > threshold: 
                RS+=1
            if i[3] > threshold: 
                RN+=1
        if i[4]==2: 
            if i[0] > threshold: 
                SW+=1
            if i[1] > threshold: 
                SR+=1
            if i[2] > threshold: 
                SS+=1
            if i[3] > threshold: 
                SN+=1
        if i[4]==3: 
            if i[0] > threshold: 
                NW+=1
            if i[1] > threshold: 
                NR+=1
            if i[2] > threshold: 
                NS+=1
            if i[3] > threshold: 
                NN+=1
    
    Confision_matrix = [[WW, WR, WS, WN], 
                        [RW, RR, RS, RN], 
                        [SW, SR, SS, SN], 
                        [NW, NR, NS, NN]]

    
    df_cm = pd.DataFrame(Confision_matrix, PredictionsLabels, PredictionsLabels)
    #print(df_cm)
    # plt.figure(figsize=(10,7))
    #pd.DataFrame.style.format(thousands=',')
    with sn.plotting_context(rc={"font.size": 12}):
        #sn.set(font_scale=1.4) # for label size
        sn.heatmap(df_cm, annot=True, cmap="Blues", annot_kws={"size": 15}, fmt='g') # font size
        plt.yticks(rotation=0) 
        plt.show()
    TP = WW + RR+ SS + NN
    FP = RW +SW + NW +WR + SR + NR + WN +RN +SN
    FN = WR +WS + WN + RW + RS + RN + SW + SR + SN + NW + NR + NS
    TNW = RR + RS + RN + SR + SS + SN + NR + NS + NN
    TNR = WW + WS + WN + SW + SS + SN + NW + NS + NN
    TNS = WW + WR + WN + RW + RR + RN + NW + NR + NN
    TNN = WW + WR + WS + RW + RR + RS + SW + SR + SS
    TN = TNW + TNR + TNS + TNN
    Accuracy = (TP + TN)/(TP + FN + FP + TN)
    Precision = TP / (TP + FP)
    Recall = TP/(TP + FN)
    F1Score = 2* ((Precision * Recall)/(Precision + Recall))
    Sensitivity = Recall
    Specificity = TN / (TN + FP)
    Balanced_accuracy = (Sensitivity + Specificity)/2
    print("Metrics: ")
    print("Accuracy = " + str(Accuracy))
    print("Balanced accuracy  = " + str(Balanced_accuracy))
    print("F1 Score = " + str(F1Score))

        





    

In [ ]:
get_f1_score()

In [ ]:
def get_f1_scorePerson(): 

    TT, TN, NT,NN = 0,0,0,0   # T-Target, N- not target, TT-true positive (Target grond truth and target predicted) etc
   

    for i in predictionsPerson:
        if i[1]==1: 
            if i[0] > threshold: 
                TT+=1
            else: 
                TN+=1
        if i[1]==0: 
            if i[0] >= threshold: #here was >=
                NT+=1
            else: 
                NN+=1        
        
      
    
    Confision_matrix = [[TT, TN], 
                        [NT, NN]]

    
    df_cm = pd.DataFrame(Confision_matrix, ["Person 16", "Other Persons"], ["Person 16", "Other Persons"])
    #print(df_cm)
    # plt.figure(figsize=(10,7))
    #pd.DataFrame.style.format(thousands=',')
    with sn.plotting_context(rc={"font.size": 12}):
        #sn.set(font_scale=1.4) # for label size
        sn.heatmap(df_cm, annot=True, cmap="Blues", annot_kws={"size": 15}, fmt='g') # font size
        plt.yticks(rotation=0) 
        plt.show()
    #sn.set(font_scale=1.4) # for label size
    #sn.heatmap(df_cm, annot=True, cmap="Blues", annot_kws={"size": 15}, fmt='g') # font size
    #plt.yticks(rotation=0) 
    #plt.show()
    TP = TT
    FP = TN
    FN = NT
    TN = NN
    Accuracy = (TP + TN)/(TP + FN + FP + TN)
    Precision = TP / (TP + FP)
    Recall = TP/(TP + FN)
    F1Score = 2* ((Precision * Recall)/(Precision + Recall))
    Sensitivity = Recall
    Specificity = TN / (TN + FP)
    Balanced_accuracy = (Sensitivity + Specificity)/2
    print("Metrics: ")
    print("Accuracy = " + str(Accuracy))
    print("Balanced accuracy  = " + str(Balanced_accuracy))
    print("F1 Score = " + str(F1Score))
    print("Precision = " + str(Precision))
    print("Recall = "+ str(Recall))


In [ ]:
#calculate F1 score for the magic wand model 

Matrix = [[82,0,0,0],
          [1,77,4,2],
          [0,0,83,5],
          [0,0,4,126]]
WW = 82
WR = 0
WS = 0
WN = 0
RW = 1
RR = 77
RS = 4
RN = 2
SW = 0
SR = 0
SS = 83
SN = 5
NW = 0
NR = 0
NS = 4
NN = 126          
Confision_matrix = [[WW, WR, WS, WN], 
                    [RW, RR, RS, RN], 
                    [SW, SR, SS, SN], 
                    [NW, NR, NS, NN]]


df_cm = pd.DataFrame(Confision_matrix, PredictionsLabels, PredictionsLabels)
#print(df_cm)
# plt.figure(figsize=(10,7))
#pd.DataFrame.style.format(thousands=',')
fig = figure(dpi=300)
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, cmap="Blues", annot_kws={"size": 15}, fmt='g') # font size
plt.yticks(rotation=0) 

TP = WW + RR+ SS + NN
FP = RW +SW + NW +WR + SR + NR + WN +RN +SN
FN = WR +WS + WN + RW + RS + RN + SW + SR + SN + NW + NR + NS
TNW = RR + RS + RN + SR + SS + SN + NR + NS + NN
TNR = WW + WS + WN + SW + SS + SN + NW + NS + NN
TNS = WW + WR + WN + RW + RR + RN + NW + NR + NN
TNN = WW + WR + WS + RW + RR + RS + SW + SR + SS
TN = TNW + TNR + TNS + TNN
Accuracy = (TP + TN)/(TP + FN + FP + TN)
Precision = TP / (TP + FP)
Recall = TP/(TP + FN)
F1Score = 2* ((Precision * Recall)/(Precision + Recall))
Sensitivity = Recall
Specificity = TN / (TN + FP)
Balanced_accuracy = (Sensitivity + Specificity)/2
print("Metrics: ")
print("Accuracy = " + str(Accuracy))
print("Balanced accuracy  = " + str(Balanced_accuracy))
print("F1 Score = " + str(F1Score))
plt.savefig('F1 original model.jpg')
plt.show()

In [ ]:
##for magic wand

for j,i in enumerate(Files):     
    if(check_serial(i)): 
        print("A datapoint has been skipped or wrongly transfered")  
    else: 
        print("Serial transmission had no issues")
    check_performance(i)
    
    check_predictions(i,j)

print(len(predictions))




In [ ]:
#for person detection
for j,i in enumerate(Files):     
    if(check_serial(i)): 
        print("A datapoint has been skipped or wrongly transfered")  
    else: 
        print("Serial transmission had no issues")
    check_performance(i)
    
    check_predictionsPerson(i,j)

print(len(predictionsPerson))



In [ ]:
def plot_graph(index): 
    Predictions0 = []
    Predictions1 = []
    Predictions2 = []
    Predictions3 = []
    for i in predictions: 
        if i[4]==index: 
            Predictions0.append(i[0])
            Predictions1.append(i[1])
            Predictions2.append(i[2])
            Predictions3.append(i[3])
    
    Wing_labels= []
    Ring_labels= []
    Slope_labels= []
    Nothing_labels= []
    Wing_labels = [i for i in Predictions0 if i>=threshold]
    Ring_labels = [i for i in Predictions1 if i>=threshold]
    Slope_labels = [i for i in Predictions2 if i>=threshold]
    Nothing_labels= [i for i in Predictions3 if i>=threshold]
    print(len(Wing_labels))

    
    Number_of_predictions = [len(Wing_labels), len(Ring_labels), len(Slope_labels), len(Nothing_labels)]
    colors = ["blue", "orange", "green", "red"]

    LabelX = range(0,len(Predictions0))
    print(len(Predictions0))


        
    #plt.figure(0)

    fig = figure(figsize=(30, 20), dpi=300)
    fig.suptitle("Predictions with ground truth data of " + PredictionsLabels[index], size=20, y=1.0)
    #fig.suptitle("Quantized Predictions with ground truth data of " + PredictionsLabels[index], size=20, y=1.0)
    All_Labels = plt.subplot2grid((5,4), (0,0),colspan=3)
    All_Labels.scatter(LabelX, Predictions0, label= "Wing", s=5, c="blue")
    All_Labels.scatter(LabelX, Predictions1,label= "Ring",s=5,c="orange")
    All_Labels.scatter(LabelX, Predictions2,label= "Slope",s=5,c="green")
    All_Labels.scatter(LabelX, Predictions3,label= "Nothing",s=5,c="red")
    All_Labels.legend(fontsize='large')
    plt.ylabel('Prediction')
    plt.xlabel('Datapoints')
    Wing = plt.subplot2grid((5,4), (1,0),colspan=3)
    Wing.scatter(LabelX, Predictions0, label= "Wing", s=5,c="blue")
    plt.ylabel('Prediction')
    plt.xlabel('Datapoints')
    Ring = plt.subplot2grid((5,4), (2,0),colspan=3)
    Ring.scatter(LabelX, Predictions1, label= "Ring", s=5, c="orange")
    plt.ylabel('Prediction')
    plt.xlabel('Datapoints')
    Slope = plt.subplot2grid((5,4), (3,0),colspan=3)
    Slope.scatter(LabelX, Predictions2, label= "Slope", s=5, c="green")
    plt.ylabel('Prediction')
    plt.xlabel('Datapoints')
    Nothing = plt.subplot2grid((5,4), (4,0),colspan=3)
    Nothing.scatter(LabelX, Predictions3, label= "Nothing", s=5, c="red")
    plt.ylabel('Prediction')
    plt.xlabel('Datapoints')
    All_labelsHist = plt.subplot2grid((5,4), (0,3))
    All_labelsHist.bar(PredictionsLabels,Number_of_predictions, color = colors, label=  PredictionsLabels)
    plt.ylabel('Datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Wing2 = plt.subplot2grid((5,4), (1,3))
    Wing2.hist(Predictions0,  label= "Wing", color="blue")
    plt.ylabel('Datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Ring2 = plt.subplot2grid((5,4), (2,3))
    Ring2.hist(Predictions1,  label= "Ring",color="orange")
    plt.ylabel('Datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Slope2 = plt.subplot2grid((5,4), (3,3))
    Slope2.hist(Predictions2,  label= "Slope",color="green")
    plt.ylabel('Datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Nothing2 = plt.subplot2grid((5,4), (4,3))
    Nothing2.hist(Predictions3,  label= "Nothing", color="red")
    plt.ylabel('Datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    
    plt.tight_layout()
    plt.show()
    plt.savefig(PredictionsLabels[index] + 'predictions NEW quantized.jpg')
  
    plt.close


In [ ]:

for i in range(4): 
    print(i)
    plot_graph(i)

In [ ]:
for i in range(4): 
    print(i)
    plot_graphPerson(i)

In [ ]:
print(predictionsPerson)

In [ ]:
def plot_graphPerson(index):   #first index is 0 
    PredictionsPerson = []           # predictions for the person to be identified
    PredictionsNotPerson = []           # predictions that the person is somebody else

    for i in predictionsPerson: 
        if i[1]==index: 
            PredictionsPerson.append(i[0])
            PredictionsNotPerson.append(round(1-float(i[0]),2))
           
    #print(PredictionsNotPerson)
    #print(PredictionsPerson)
    Target_labels= []
    NotTarget_labels= []
    
    Target_labels = [i for i in PredictionsPerson if i>=threshold]           #all the predictions predict the person with a probabiltiy >= 0.5
    NotTarget_labels = [i for i in PredictionsNotPerson if i>threshold]        #all the predictions that predict somebody else with a probabiltiy > 0.5
    
    print(len(Target_labels))

    
    Number_of_predictionsPerson = [len(Target_labels), len(NotTarget_labels)]
    colors = ["blue", "red"]

    LabelX = range(0,len(PredictionsPerson))
    print(len(PredictionsPerson))

    
        
    #plt.figure(0)

    fig = figure(figsize=(30, 20), dpi=300)
    fig.suptitle("Predictions with ground truth data of " + predictionsLabelsPerson[index], size=20, y=1.0)  ##check this
    #fig.suptitle("Quantized Predictions with ground truth data of " + PredictionsLabels[index], size=20, y=1.0)
    All_Labels = plt.subplot2grid((5,4), (0,0),colspan=3)
    All_Labels.scatter(LabelX, PredictionsPerson, label= "Target", s=5, c="blue")
    All_Labels.scatter(LabelX, PredictionsNotPerson,label= "Not Target",s=5,c="red")
  
    #All_Labels.legend(fontsize='large')
    plt.ylabel('Prediction')
    plt.xlabel('Batch of 50 datapoints')
    Target = plt.subplot2grid((5,4), (1,0),colspan=3)
    Target.scatter(LabelX, PredictionsPerson, label= "Target", s=5,c="blue")
    plt.ylabel('Prediction')
    plt.xlabel('Batch of 50 datapoints')
    Not_Target = plt.subplot2grid((5,4), (2,0),colspan=3)
    Not_Target.scatter(LabelX, PredictionsNotPerson, label= "Not Target", s=5, c="red")
    plt.ylabel('Prediction')
    plt.xlabel('Batch of 50 datapoints')

    All_labelsHist = plt.subplot2grid((5,4), (0,3))
    All_labelsHist.bar(["Person 16", "Other persons"],Number_of_predictionsPerson, color = colors)
    plt.ylabel('Batch of 50 datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Target2 = plt.subplot2grid((5,4), (1,3))
    Target2.hist(PredictionsPerson,  label= "Target", color="blue")
    plt.ylabel('Batch of 50 datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    Not_Target2 = plt.subplot2grid((5,4), (2,3))
    Not_Target2.hist(PredictionsNotPerson,  label= "Not Target",color="red")
    plt.ylabel('Batch of 50 datapoints')
    plt.xlabel('Predictions bigger than '+ str(threshold))
    
    plt.grid(False)
    plt.tight_layout()
    plt.show()
    plt.savefig(predictionsLabelsPerson[index] + 'Person detection quantized try2.jpg')
    
    plt.close